In [1]:
from pyspark import SQLContext
from pyspark import HiveContext
list_A = [{'name':'Karthikeyan','id':10},{'name':'Navaneethan','id':20}]
list_B = [{'id':10,'add1':'DE'},{'id':30,'add1':'NJ'},{'id':40,'add1':'NY'}]
df_A = spark.createDataFrame(list_A)
df_B = spark.createDataFrame(list_B)
df_A.show()
df_B.show()

+---+-----------+
 id| name|
+---+-----------+
 10|Karthikeyan|
 20|Navaneethan|
+---+-----------+

+----+---+
add1| id|
+----+---+
 DE| 10|
 NJ| 30|
 NY| 40|
+----+---+

In [2]:
df_A.crossJoin(df_B).show()
df_B.crossJoin(df_A).show()

+---+-----------+----+---+
 id| name|add1| id|
+---+-----------+----+---+
 10|Karthikeyan| DE| 10|
 10|Karthikeyan| NJ| 30|
 10|Karthikeyan| NY| 40|
 20|Navaneethan| DE| 10|
 20|Navaneethan| NJ| 30|
 20|Navaneethan| NY| 40|
+---+-----------+----+---+

+----+---+---+-----------+
add1| id| id| name|
+----+---+---+-----------+
 DE| 10| 10|Karthikeyan|
 DE| 10| 20|Navaneethan|
 NJ| 30| 10|Karthikeyan|
 NJ| 30| 20|Navaneethan|
 NY| 40| 10|Karthikeyan|
 NY| 40| 20|Navaneethan|
+----+---+---+-----------+

In [3]:
#inner ,outer 
df_A.join(df_B,df_A.id == df_B.id,'inner').show()
df_A.join(df_B,df_A.id == df_B.id,'outer').show()
df_A.join(df_B,df_A.id == df_B.id,'full').show()
df_A.join(df_B,df_A.id == df_B.id,'left').show()
df_A.join(df_B,df_A.id == df_B.id,'left_outer').show()
df_A.join(df_B,df_A.id == df_B.id,'right').show()
df_A.join(df_B,df_A.id == df_B.id,'left_semi').show()
df_A.join(df_B,df_A.id == df_B.id,'left_anti').show()

+---+-----------+----+---+
 id| name|add1| id|
+---+-----------+----+---+
 10|Karthikeyan| DE| 10|
+---+-----------+----+---+

+----+-----------+----+----+
 id| name|add1| id|
+----+-----------+----+----+
 10|Karthikeyan| DE| 10|
null| null| NJ| 30|
 20|Navaneethan|null|null|
null| null| NY| 40|
+----+-----------+----+----+

+----+-----------+----+----+
 id| name|add1| id|
+----+-----------+----+----+
 10|Karthikeyan| DE| 10|
null| null| NJ| 30|
 20|Navaneethan|null|null|
null| null| NY| 40|
+----+-----------+----+----+

+---+-----------+----+----+
 id| name|add1| id|
+---+-----------+----+----+
 10|Karthikeyan| DE| 10|
 20|Navaneethan|null|null|
+---+-----------+----+----+

+---+-----------+----+----+
 id| name|add1| id|
+---+-----------+----+----+
 10|Karthikeyan| DE| 10|
 20|Navaneethan|null|null|
+---+-----------+----+----+

+----+-----------+----+---+
 id| name|add1| id|
+----+-----------+----+---+
 10|Karthikeyan| DE| 10|
null| null| NJ| 30|
null| null| NY| 40|
+----+-----------+----+---+

+---+-----------+
 id| name|
+---+-----------+
 10|Karthikeyan|
+---+-----------+

+---+-----------+
 id| name|
+---+-----------+
 20|Navaneethan|
+---+-----------+

In [4]:
#hint
df_A.join(df_B,df_A.id == df_B.id,'inner').explain()

== Physical Plan ==
*(3) SortMergeJoin [id#956L], [id#961L], Inner
:- Sort [id#956L ASC NULLS FIRST], false, 0
: +- Exchange hashpartitioning(id#956L, 200)
: +- *(1) Filter isnotnull(id#956L)
: +- *(1) Scan ExistingRDD[id#956L,name#957]
+- Sort [id#961L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#961L, 200)
 +- *(2) Filter isnotnull(id#961L)
 +- *(2) Scan ExistingRDD[add1#960,id#961L]

In [5]:
df_A.join(df_B.hint('broadcast'),'id').explain()

== Physical Plan ==
*(2) Project [id#956L, name#957, add1#960]
+- *(2) BroadcastHashJoin [id#956L], [id#961L], Inner, BuildRight
 :- *(2) Filter isnotnull(id#956L)
 : +- *(2) Scan ExistingRDD[id#956L,name#957]
 +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, bigint, false]))
 +- *(1) Filter isnotnull(id#961L)
 +- *(1) Scan ExistingRDD[add1#960,id#961L]

In [6]:
#repartition
df_res = df_A.crossJoin(df_B.select("add1")).select("ID","name","add1")
df_res.show()
df_res.repartition(2,"add1").show()
df_res.repartition("add1").show()

+---+-----------+----+
 ID| name|add1|
+---+-----------+----+
 10|Karthikeyan| DE|
 10|Karthikeyan| NJ|
 10|Karthikeyan| NY|
 20|Navaneethan| DE|
 20|Navaneethan| NJ|
 20|Navaneethan| NY|
+---+-----------+----+

+---+-----------+----+
 ID| name|add1|
+---+-----------+----+
 10|Karthikeyan| NJ|
 10|Karthikeyan| NY|
 20|Navaneethan| NJ|
 20|Navaneethan| NY|
 10|Karthikeyan| DE|
 20|Navaneethan| DE|
+---+-----------+----+

+---+-----------+----+
 ID| name|add1|
+---+-----------+----+
 10|Karthikeyan| NJ|
 20|Navaneethan| NJ|
 10|Karthikeyan| DE|
 20|Navaneethan| DE|
 10|Karthikeyan| NY|
 20|Navaneethan| NY|
+---+-----------+----+

In [7]:
#cross tab
df_A.crosstab('id','name').show()

+-------+-----------+-----------+
id_name|Karthikeyan|Navaneethan|
+-------+-----------+-----------+
 20| 0| 1|
 10| 1| 0|
+-------+-----------+-----------+

In [8]:
# 3 Dimension 
from pyspark import SQLContext
from pyspark import HiveContext
list_A = [{'name':'Karthikeyan','id':10,'age':34},{'name':'Navaneethan','id':20,'age':45}]
df_A =spark.createDataFrame(list_A)
df_A.show()
df_A.crosstab('age','id').show()
df_A.crosstab('id','age').show()

+---+---+-----------+
age| id| name|
+---+---+-----------+
 34| 10|Karthikeyan|
 45| 20|Navaneethan|
+---+---+-----------+

+------+---+---+
age_id| 10| 20|
+------+---+---+
 34| 1| 0|
 45| 0| 1|
+------+---+---+

+------+---+---+
id_age| 34| 45|
+------+---+---+
 20| 0| 1|
 10| 1| 0|
+------+---+---+

In [9]:
df_A.show()
df_A.cube('id','name').count().show()

+---+---+-----------+
age| id| name|
+---+---+-----------+
 34| 10|Karthikeyan|
 45| 20|Navaneethan|
+---+---+-----------+

+----+-----------+-----+
 id| name|count|
+----+-----------+-----+
null| null| 2|
 10| null| 1|
null|Karthikeyan| 1|
 20| null| 1|
 10|Karthikeyan| 1|
 20|Navaneethan| 1|
null|Navaneethan| 1|
+----+-----------+-----+

In [10]:
df_A.rollup("ID","name").count().show()

+----+-----------+-----+
 ID| name|count|
+----+-----------+-----+
null| null| 2|
 10| null| 1|
 20| null| 1|
 10|Karthikeyan| 1|
 20|Navaneethan| 1|
+----+-----------+-----+

In [11]:
tmp_list = [('A',10),('B',20)]
df = spark.createDataFrame(tmp_list)
df.storageLevel


Out[ 52 ]: StorageLevel(False, False, False, False, 1)

In [12]:
df.cache().storageLevel

Out[ 53 ]: StorageLevel(True, True, False, True, 1)

In [13]:
from pyspark.storagelevel import StorageLevel 

df.persist(StorageLevel.DISK_ONLY_2).storageLevel

Out[ 54 ]: StorageLevel(True, True, False, True, 1)

In [14]:
df.persist(StorageLevel.MEMORY_ONLY).storageLevel

Out[ 55 ]: StorageLevel(True, True, False, True, 1)